In [30]:
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid
from matplotlib import pyplot as plt
import statsmodels.api as sm

In [25]:
data = pd.read_stata('/Users/cassturk/Downloads/angrist-krueger91.dta')

In [ ]:
#checking replication for OLS

j =pd.get_dummies(data['yob'])

m = data['yob'].astype(str)+data['qob'].astype(str)
l = pd.get_dummies(m)

d = np.array([list(data['logwage'])])
a= np.array([list(data['edu'])])
c = np.array([list(data['qob'])])

c = np.r_[np.array(l).T, np.array(j).T]

b = np.r_[a,np.array(j).T]

beta_ols = np.linalg.solve(b@b.T, b@d.T)

#note it's pretty good:
print(beta_ols)

In [34]:
data1 = pd.read_stata('/Users/cassturk/Downloads/angrist-krueger91.dta')

oo = 0
for i in range(2000):
    #my computer can't compute the whole projection matrix so here's some sampling lol
    data = data1.sample(n=10000)
    
    #creating dummies
    j =pd.get_dummies(data['yob'])
    k =pd.get_dummies(data['qob'])

    m = data['yob'].astype(str)+data['qob'].astype(str)

    l = pd.get_dummies(m)

    #dropping linear combinations

    l = l.drop(columns=['19393', '19391', '19392', '19394', '19303', '19313', '19323', 
                        '19333', '19343','19353', '19363','19373','19383'])

    j = j.drop(columns = [1939])

    k = k.drop(columns = [3])


    #creating matrices
    y = np.array([list(data['logwage'])]).squeeze()
    a = np.array([list(data['edu'])])

    x = np.r_[np.array([list(data['edu'])]),np.array(j).T].squeeze()

    #computation heavy stuff
    z = np.r_[np.array(l).T, np.array(j).T]
    z = z.astype('float32')
    zint = np.linalg.pinv(z@z.T)
    
    #checking with canned functions
    model = sm.OLS(a.T,z.T)
    results = model.fit()
    reses = results.params
    
    xhat = reses@z
    
    xs = np.r_[[xhat.T], np.array(j).T]
    model = sm.OLS(y.T,xs.T)
    results = model.fit()
    #print(results.params[0])
    #canned gives same answers as both paper and samples

    #the really computation-heavy stuff, according to where my computer chooses to die most frequently
    zpr = z.T@zint
    zproj = zpr@z
    xzpr =x@zproj
    xzprx = xzpr@x.T
    zpry = xzpr@y.T
    beta_iv = np.linalg.solve(xzprx, zpry)
    beta_iv_x = beta_iv.squeeze()
    oo= oo+beta_iv_x[0]
    print(beta_iv_x[0])
print('average')
print(oo/2000)

0.026187830779796584
0.026187964474636307
-0.009559639717861446
-0.009559651506611255
0.08945470753807189
0.08945478677121156
0.09023534824922208
0.09023522455823255
0.11961922150043819
0.11961950715272474
0.04429963070129428
0.044299577872634006
0.0944212803871856
0.09442143486391248
0.03070466327500318
0.03070459344946319
0.10748914652838093
0.10748923816753947
0.07694002148082729
0.07694007977606768
0.0458457494231731
0.045845750375946015
0.07401162232686173
0.07401167296277279
0.04458505014094083
0.044585142937566234
0.04037441956916141
0.04037422413435029
0.08454658765670003
0.08454677178521995
0.0478220211113399
0.047821908927847344
0.060870437084512174
0.060870484774372485
0.04462970272342384
0.04462975380954493
0.1327288705782353
0.13272880020620317
0.03881862007357212
0.03881861919578466
average
0.0006420129423447093
